# **Sentiment Analysis Project**

---

## 개요
- NLP(Natural Language Processing)
- ~~LSTM (Long Short-Term Memory)~~
- IMDB 영화 리뷰 dataset을 학습데이터로 사용
- 각 리뷰를 별점을 기준으로 positive, negative, neutural로 분류 
- 특정 영화에 대한 리뷰를 분석하여 해당 영화에 대한 관람객의 평가를 도출 
<br>(긍정: x %, 부정: y%)
- 새로운 리뷰가 작성되었을 때 종합 평가에 즉각적으로 반영

## 활용
- 별점이 없는 리뷰에 대해서도 예상 별점을 책정할 수 있음 
- 별점과 리뷰가 서로 상이한 리뷰에 대해서 필터링을 통해 비교적 더 정확한 평가를 제공할 수 있음
- 영화에 대한 리뷰를 통해 개별 추천 시스템을 고도화 할 수 있음   

### 라이브러리 불러오기

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

### 데이터 셋 다운로드 및 추출

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

# 데이터 셋 경로
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
train_dir = os.path.join(dataset_dir, 'train')
# /neg : 부정적인 리뷰 txt파일 모음
# /pos : 긍정적인 리뷰 txt파일 모음

# 사용 하지 않는 폴더 제거
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

###train, test, validation으로 데이터셋 분리
현재 train, test 각각 25000개의 데이터가 있고 train데이터를 train과 validation으로 분리 (80:20)

In [ ]:
batch_size = 32
seed = 42

train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


성능 향상
- 디스크에서 로드 후 메모리에 데이터 보관
- 훈련 중에 데이터와 전처리와 모델 실행을 병행

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

### 벡터화 레이어

In [ ]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')  # html 태그 제거
  return tf.strings.regex_replace(stripped_html,                      # 모든 구두점 제거
                                  '[%s]' % re.escape(string.punctuation),
                                  '')
  
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

train_text = train_ds.map(lambda x, y: x) # 전처리 레이어의 상태를 데이터세트에 맞춤
vectorize_layer.adapt(train_text)

### 모델 생성

In [ ]:
embedding_dim = 16

model = tf.keras.Sequential([
  vectorize_layer,  
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1, activation='sigmoid')
  ])

### 손실함수와 옵티마이저 적용 

In [ ]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=False),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

### 모델 훈련

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


625/625 [==============================] - 9s 14ms/step - loss: 0.6651 - binary_accuracy: 0.4981 - val_loss: 0.6160 - val_binary_accuracy: 0.5076
Epoch 2/10
625/625 [==============================] - 7s 12ms/step - loss: 0.5498 - binary_accuracy: 0.4981 - val_loss: 0.4989 - val_binary_accuracy: 0.5076
Epoch 3/10
625/625 [==============================] - 9s 14ms/step - loss: 0.4458 - binary_accuracy: 0.4981 - val_loss: 0.4207 - val_binary_accuracy: 0.5076
Epoch 4/10
625/625 [==============================] - 9s 14ms/step - loss: 0.3789 - binary_accuracy: 0.4981 - val_loss: 0.3743 - val_binary_accuracy: 0.5076
Epoch 5/10
625/625 [==============================] - 7s 11ms/step - loss: 0.3361 - binary_accuracy: 0.4981 - val_loss: 0.3454 - val_binary_accuracy: 0.5076
Epoch 6/10
625/625 [==============================] - 8s 13ms/step - loss: 0.3054 - binary_accuracy: 0.4981 - val_loss: 0.3264 - val_binary_accuracy: 0.5076
Epoch 7/10
625/625 [==============================] - 8s 14ms/step - 

### 모델 평가

In [ ]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 [==============================] - 5s 6ms/step - loss: 0.3102 - binary_accuracy: 0.5000
Loss:  0.3101525604724884
Accuracy:  0.5


### 사용자 입력 테스트

In [ ]:
review = [(input('Movie Review: '))]
result = model.predict(review)
if result >= 0.5:
  print('긍정적인 리뷰입니다!')
else:
  print('부정적인 리뷰입니다!')

Movie Review: so fun nice!
1/1 [==============================] - 0s 59ms/step
긍정적인 리뷰입니다!
